Place de marché
==============

![logo](../reports/figures/logo.png)


### Votre mission
Votre mission est de **réaliser une première étude de faisabilité d'un moteur de classification** d'articles basé sur une image et une description pour l'automatisation de l'attribution de la catégorie de l'article.

Pour ce faire, vous allez **évaluer la possibilité d'extraire des données depuis l'API Amazon** en **prenant connaissance de la documentation** et en **écrivant la requête** qui vous permettrait d'extraire des données supplémentaires. Vous vous assurerez ainsi que vous pourrez bien disposer de plus de données et diversifier les sources de données pour éviter les biais pour votre moteur de classification.

Ensuite, vous **analyserez le jeu de données** déjà constitué en **réalisant un prétraitement** des images et des descriptions des produits, une **réduction de dimension**, puis un **clustering**. Les résultats du clustering seront présentés sous la forme d’une représentation en deux dimensions à déterminer, qui ’illustrera le fait que les caractéristiques extraites permettent de regrouper des produits de même catégorie.

La représentation graphique vous aidera à convaincre Linda que cette approche de modélisation permettra bien de regrouper des produits de même catégorie.

### Contraintes

Linda vous a communiqué les contraintes suivantes :

   * Limiter le nombre d’articles pris par l’API (par exemple : 1000 lignes) et filtrer sur un unique type d’article (par exemple un type d’article peu présent dans votre échantillon de données actuelles).
   * Afin d’extraire les features, mettre en œuvre a minima un algorithme de type SIFT / ORB / SURF.
   * Un algorithme de type CNN Transfer Learning peut éventuellement être utilisé en complément, s’il peut apporter un éclairage supplémentaire à la démonstration.

In [ ]:
import os
import random

import numpy as np
import cv2
from PIL import Image, ImageOps, ImageFilter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans

import plotly.express as px


sns.set(font_scale=1.6)

Image.MAX_IMAGE_PIXELS = 93680329


### Chargement des descriptions

In [ ]:
for dirname, _, filenames in os.walk('../data/raw/'):
    if len(filenames) == 1:
        df = pd.read_csv(os.path.join(dirname, filenames[0]))

In [ ]:
df['path'] = df['image'].apply(lambda x: os.path.join('../data/raw/Images/', x))

In [ ]:
col_to_drop = [
    'uniq_id',
    'crawl_timestamp',
    'product_url',
    'pid',
    'discounted_price',
    'is_FK_Advantage_product',
    'product_rating',
    'overall_rating',
    'product_specifications',
    'brand',
    
]
df.drop(columns=col_to_drop, inplace=True)

In [ ]:
def extract_level(tree_str, level=-1, strict=False):
    """return a specific level from product_category_tree.
    tips: specify a negative index to access latest part of the tree.
    """
    tree_str = eval(tree_str)[0]
    levels = tree_str.split('>>')
    levels = list(map(lambda x: x.strip(), levels))
    if strict:
        return levels[level]
    else:
        try:
            return levels[level]
        except IndexError:
            return None

On récupère le premier niveau de l'arbre des catégories comme label

In [ ]:
df['label'] = df['product_category_tree'].apply(extract_level, level=1)

In [ ]:
df['label'].unique().shape

### Resnet50 comme extracteur de features

In [ ]:
from tensorboard.plugins import projector

import keras
from keras.applications.resnet50 import ResNet50 

In [ ]:
base_model = ResNet50(weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
model = keras.Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
image_224 = np.array([np.array(Image.open(x).resize((224, 224))) for x in df['path']])

In [ ]:
image_224.shape

In [ ]:
features = model.predict(image_224)

In [ ]:
features.shape

In [ ]:
tsne = TSNE(n_components=2, perplexity=9, learning_rate=10, n_iter=2500)
tsne_res = tsne.fit_transform(features)

In [ ]:
tsne_res = pd.DataFrame(tsne_res)
tsne_res['label'] = df['label']

In [ ]:
tsne_res

In [ ]:
centers = tsne_res.groupby('label').mean()
centers.reset_index(drop=False, inplace=True)

In [ ]:
centers

In [ ]:
_, ax = plt.subplots(1, figsize=(24, 18))
palette = sns.color_palette(None, centers.shape[0])
for i, center in enumerate(centers['label']):
    if tsne_res.groupby('label').count().loc[center, 0] > 10:
        ax.scatter(x=tsne_res.loc[tsne_res['label'] == center, 0],
           y=tsne_res.loc[tsne_res['label'] == center, 1],
           color=palette[i]
          )
        
        ax.annotate(center, centers.set_index('label').loc[center, :].values,
                    color=palette[i]
                   )
# plt.axis('off')
plt.show()

In [ ]:
Image.fromarray(255 - np.array(image_224[7]))

In [ ]:
Image.fromarray(255 - np.array(image_224[45]))

In [ ]:
Image.fromarray(np.array(image_224[45]))

### Visualisation dans TensorBoard

**warning** Une fois le comportement de TensorFlow v2 désactivé, Keras va lever une exception!


In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [ ]:
 def images_to_sprite(data):
        """Creates the sprite image along with any necessary padding

        Args:
          data: NxHxW[x3] tensor containing the images.

        Returns:
          data: Properly shaped HxWx3 image with any necessary padding.
        """
        if len(data.shape) == 3:
            data = np.tile(data[...,np.newaxis], (1,1,1,3))
        data = data.astype(np.float32)
        min = np.min(data.reshape((data.shape[0], -1)), axis=1)
        data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
        max = np.max(data.reshape((data.shape[0], -1)), axis=1)
        data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
        # Inverting the colors seems to look better for MNIST
        #data = 1 - data

        n = int(np.ceil(np.sqrt(data.shape[0])))
        padding = ((0, n ** 2 - data.shape[0]), (0, 0),
                (0, 0)) + ((0, 0),) * (data.ndim - 3)
        data = np.pad(data, padding, mode='constant',
                constant_values=0)
        # Tile the individual thumbnails into an image.
        data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
                + tuple(range(4, data.ndim + 1)))
        data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
        data = (data * 255).astype(np.uint8)
        return data

In [ ]:
Image.fromarray(images_to_sprite(image_224))

In [ ]:
tf_data = tf.Variable(features, name='features')

LOG_DIR = '../reports/tf/sessions/resnet/'

sprite = images_to_sprite(image_224)
cv2.imwrite(os.path.join(LOG_DIR, 'sprite_4_classes.png'), sprite)

metadata = 'df_labels.tsv'

# df[['image', 'label']].to_csv(os.path.join(LOG_DIR, metadata), index=False, header=False)
df['label'].to_csv(os.path.join(LOG_DIR, metadata), index=False, header=False)


In [ ]:
# with tf.Session() as sess:
#     saver = tf.train.Saver([tf_data])
#     sess.run(tf_data.initializer)
#     saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
#     config = projector.ProjectorConfig()
    
#     embedding = config.embeddings.add()
#     embedding.tensor_name = tf_data.name
    
#     embedding.metadata_path = metadata
    
#     embedding.sprite.image_path = 'sprite_4_classes.png'
#     embedding.sprite.single_image_dim.extend([image_224.shape[1], image_224.shape[1]])

#     projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), 
#                                    config)